In [1]:
import pandas as pd
import json
from collections import defaultdict
from tqdm import tqdm
import random
from copy import deepcopy
import re


SPIECE_UNDERLINE = '▁'
def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F or c == SPIECE_UNDERLINE:
        return True
    return False

def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a peice of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens
    
def _is_chinese_char(cp):
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
            (cp >= 0x3400 and cp <= 0x4DBF) or  #
            (cp >= 0x20000 and cp <= 0x2A6DF) or  #
            (cp >= 0x2A700 and cp <= 0x2B73F) or  #
            (cp >= 0x2B740 and cp <= 0x2B81F) or  #
            (cp >= 0x2B820 and cp <= 0x2CEAF) or
            (cp >= 0xF900 and cp <= 0xFAFF) or  #
            (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
        return True

    return False

def is_fuhao(c):
    if c == '。' or c == '，' or c == '！' or c == '？' or c == '；' or c == '、' or c == '：' or c == '（' or c == '）' \
            or c == '－' or c == '~' or c == '「' or c == '《' or c == '》' or c == ',' or c == '」' or c == '"' or c == '“' or c == '”' \
            or c == '$' or c == '『' or c == '』' or c == '—' or c == ';' or c == '。' or c == '(' or c == ')' or c == '-' or c == '～' or c == '。' \
            or c == '‘' or c == '’':
        return True
    return False
    
def _tokenize_chinese_chars(text):
    """Adds whitespace around any CJK character."""
    output = []
    for char in text:
        cp = ord(char)
        if _is_chinese_char(cp) or is_fuhao(char):
            if len(output) > 0 and output[-1] != SPIECE_UNDERLINE:
                output.append(SPIECE_UNDERLINE)
            output.append(char)
            output.append(SPIECE_UNDERLINE)
        else:
            output.append(char)
    return "".join(output)

def read_kg(kg_path,kg):
    #'/home/xhsun/NLP/KGQA/KG/kgCLUE/Knowledge.txt'
    with open(kg_path) as f:
        lines=f.readlines()

    print('The number of triples: {}'.format(len(lines)))

    sub_map = defaultdict(set)#每一个头实体作为key，对应的所有一跳路径内的(关系，尾实体)作为value
    # so_map=defaultdict(set)#每一对(头实体，尾实体)作为key，对应的所有一跳的关系作为value
    sp_map=defaultdict(set)#每一个（头实体，关系）作为key，对应的仅能有一个答案

    alias_map=defaultdict(set)
    ent_to_relations=defaultdict(set)
    bad_line=0
    spliter='|||' if kg=='nlpcc' else '\t'
    for i in tqdm(range(len(lines))):
        line=lines[i]
        l = line.strip().split(spliter)
        s = l[0].strip()
        p = l[1].strip()
        o = l[2].strip()
        if s=='' or p=='' or o=='':
            bad_line+=1
            continue
        sub_map[s].add((p, o))
    #     so_map[(s,o)].add(p)
        sp_map[(s,p)].add(o)

        ent_to_relations[s].add(p)

        entity_mention=s
        if kg.lower()=='kgclue' and ('（' in s and '）' in s):
            entity_mention=s.split('（')[0]
            alias_map[entity_mention].add(s)
        if kg.lower()=='nlpcc' and ('(' in s and ')' in s):
            entity_mention=s.split('(')[0]
            alias_map[entity_mention].add(s)

        if p in ['别名','中文名','英文名','昵称','中文名称','英文名称','别称','全称','原名']:
            alias_map[entity_mention].add(o)
    return alias_map,sub_map

In [2]:
alias_map,sub_map=read_kg('/home/xhsun/NLP/KGQA/KG/kgCLUE/Knowledge.txt',kg='kgclue')

The number of triples: 22883549


100%|███████████████████████████████████████████████████████████████████████████████████████| 22883549/22883549 [01:05<00:00, 348867.28it/s]


In [111]:
def read_data(data_path):
    with open(data_path) as f:
        lines=f.readlines()
    data=[]
    for line in lines:
        data.append(json.loads(line))
    print("原始数据有{}个样本".format(len(data)))
    return data

examples=read_data('../../data/kgclue/test_public.json')

原始数据有2000个样本


In [123]:
def fun(examples,sub_map):
    ner_examples=[]
    for example in examples:
        triple=example['answer']
        head,rel,tail=triple.split('|||')
        head=head.strip()
        try:
            assert head in sub_map
            relations=[r for r,t in sub_map[head]]
            example.update({"relations":relations})
            ner_examples.append(example)
        except:
            print(head,example)
            #raise Exception("check")
            continue
        
    ner_examples2=[]
    for example in ner_examples:
        triple=example['answer']
        #head,rel,tail=triple.split('|||')
        relations=example['relations']
        tag='O'
        for rel in relations:
            found_each_rel=False
            for ner_tag,tag_attrs in tag2attrs.items():
                if rel in tag_attrs:
                    found_each_rel=True
                    break
            if found_each_rel:
                tag=ner_tag
                break
        example.update({'tag':tag})
        ner_examples2.append(example)
    
    outputs=[]
    filtered_count=0
    for example in ner_examples2:
        question=example['question']
        triple=example['answer']
        tag=example['tag']
        head=triple.split('|||')[0].strip()
        if '（' in head and '）' in head:
            head=head.split('（')[0]
        if head not in question:
            filtered_count+=1
            continue
        assert head in question
        for tag_ner in tag2attrs.keys():
            input_=question+'。这个句子中有哪些属于{}？'.format(tag_ner)
            output_=head if tag_ner==tag else "空实体"
            outputs.append([input_,output_])
        if tag=='O':
            outputs.append([question+'。这个句子中有哪些属于{}？'.format('普通实体'),head])
        else:
            outputs.append([question+'。这个句子中有哪些属于{}？'.format('普通实体'),'空实体'])
    print(filtered_count)
    return outputs

In [124]:
outputs=fun(examples,sub_map)

0


In [125]:
len(outputs)

22000

In [129]:
outputs[:1000]

[['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于组织机构？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于人名？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于音乐？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于节日？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于地名？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于书籍？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于娱乐，影视节目？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于游戏？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于化学药品？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于疾病？', '空实体'],
 ['你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于普通实体？', '长江武汉航道局'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于组织机构？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于人名？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于音乐？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于节日？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于地名？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于书籍？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于娱乐，影视节目？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于游戏？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于化学药品？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于疾病？', '空实体'],
 ['东瓯王发生的主要事件是什么？。这个句子中有哪些属于普通实体？', '东瓯王'],
 ['山西省

In [88]:
examples[0]

{'id': 0,
 'question': '你知道长江武汉航道局管多少个地方吗？',
 'answer': '长江武汉航道局 ||| 管辖 ||| 715.2公里'}

In [89]:
ner_examples=[]
for example in examples:
    triple=example['answer']
    head,rel,tail=triple.split('|||')
    head=head.strip()
    try:
        assert head in sub_map
        relations=[r for r,t in sub_map[head]]
        example.update({"relations":relations})
        ner_examples.append(example)
    except:
        print(head,example)
        #raise Exception("check")
        continue
    

In [90]:
print(ner_examples[0])

{'id': 0, 'question': '你知道长江武汉航道局管多少个地方吗？', 'answer': '长江武汉航道局 ||| 管辖 ||| 715.2公里', 'relations': ['职责', '中文名', '维护管理', '管辖']}


In [105]:
tag2attrs={"组织机构":['成立日期','创始人','总部地点','员工数','成立时间','公司类型','公司名称','创办时间','创建时间','创立','所属地区','机构特点','建立时间','机构地址'],
"人名":['国籍','毕业院校','职业','出生日期','专业方向','身高','民族','血型','体重','逝世日期','性别'],
"音乐":['填词','音乐风格','所属专辑','歌曲原唱','歌曲时长','编曲','谱曲','歌手','专辑'],
"节日":['节日类型','节日意义','节日起源','节日饮食','节日活动'],
"地名":['修建时间','所在地点','著名景点','地理位置','面积','人口数量','重建时间','营业时间','人口数','总人口'],
"书籍":['装帧','出版社','丛书','作者','出版时间','书名'],
"娱乐，影视节目":['制片人','播出时间','首播时间','片长','对白语言','上映时间','主演','票房','编剧','制片地区','首演时间','集数','播出时长'],
"游戏":['游戏类型','游戏画面','游戏引擎','游戏平台','游戏类别','所属游戏','游戏大小'],
"化学药品":['用法用量'],
"疾病":['症状','传染性','相关疾病']
}

In [106]:
ner_examples2=[]
for example in ner_examples:
    triple=example['answer']
    #head,rel,tail=triple.split('|||')
    relations=example['relations']
    tag='O'
    for rel in relations:
        found_each_rel=False
        for ner_tag,tag_attrs in tag2attrs.items():
            if rel in tag_attrs:
                found_each_rel=True
                break
        if found_each_rel:
            tag=ner_tag
            break
    example.update({'tag':tag})
    ner_examples2.append(example)

In [107]:
O_count=0
for example in ner_examples2:
    if example['tag']=='O':
        O_count+=1
        print(example)

{'id': 0, 'question': '你知道长江武汉航道局管多少个地方吗？', 'answer': '长江武汉航道局 ||| 管辖 ||| 715.2公里', 'relations': ['职责', '中文名', '维护管理', '管辖'], 'tag': 'O'}
{'id': 1, 'question': '东瓯王发生的主要事件是什么？', 'answer': '东瓯王 ||| 主要事件 ||| 抗秦反秦，助汉击楚。', 'relations': ['主要事件', '中文名', '所属地', '姓氏', '东瓯王后', '代表人物'], 'tag': 'O'}
{'id': 2, 'question': '山西省铸造新工艺工程技术研究中心是什么的管辖的啊？', 'answer': '山西省铸造新工艺工程技术研究中心 ||| 机构类型 ||| 研究开发机构', 'relations': ['机构类型', '隶属', '研究项目', '中文名'], 'tag': 'O'}
{'id': 4, 'question': '国民军参与过什么战争？', 'answer': '国民军 ||| 参与战争 ||| 直奉战争、北伐战争、中原大战、抗日战争', 'relations': ['外文名', '别名', '所属派系', '改编时间', '首领', '中文名', '主要将领', '参与战争', '外文名', '领袖', '分化之前'], 'tag': 'O'}
{'id': 5, 'question': '你知道诺基亚lumia1320的主屏尺寸是多少吗？', 'answer': '诺基亚lumia1320 ||| 主屏尺寸 ||| 6英寸', 'relations': ['主屏尺寸', '中文名', '型号', '品牌', '电池容量', '核心数', '代号', '像素', '外文名'], 'tag': 'O'}
{'id': 6, 'question': '你什么时候参加的执业兽医资格考试啊？', 'answer': '执业兽医资格考试 ||| 考试时间 ||| 10月', 'relations': ['实质', '外文名', '中文名', '分类', '考试时间'], 'tag': 'O'}
{'id': 7, 'question': '你知道2007年度北京

In [108]:
O_count

994

In [109]:
len(ner_examples2)

2000

In [128]:
ner_examples2[:100]

[{'id': 0,
  'question': '你知道长江武汉航道局管多少个地方吗？',
  'answer': '长江武汉航道局 ||| 管辖 ||| 715.2公里',
  'relations': ['职责', '中文名', '维护管理', '管辖'],
  'tag': 'O'},
 {'id': 1,
  'question': '东瓯王发生的主要事件是什么？',
  'answer': '东瓯王 ||| 主要事件 ||| 抗秦反秦，助汉击楚。',
  'relations': ['主要事件', '中文名', '所属地', '姓氏', '东瓯王后', '代表人物'],
  'tag': 'O'},
 {'id': 2,
  'question': '山西省铸造新工艺工程技术研究中心是什么的管辖的啊？',
  'answer': '山西省铸造新工艺工程技术研究中心 ||| 机构类型 ||| 研究开发机构',
  'relations': ['机构类型', '隶属', '研究项目', '中文名'],
  'tag': 'O'},
 {'id': 3,
  'question': '你知道谁是丘堤的儿媳吗',
  'answer': '丘堤 ||| 儿媳 ||| 籍虹',
  'relations': ['出生日期',
   '孙女',
   '中文名',
   '出生地',
   '儿子',
   '别名',
   '主要成就',
   '逝世日期',
   '丈夫',
   '儿媳'],
  'tag': '人名'},
 {'id': 4,
  'question': '国民军参与过什么战争？',
  'answer': '国民军 ||| 参与战争 ||| 直奉战争、北伐战争、中原大战、抗日战争',
  'relations': ['外文名',
   '别名',
   '所属派系',
   '改编时间',
   '首领',
   '中文名',
   '主要将领',
   '参与战争',
   '外文名',
   '领袖',
   '分化之前'],
  'tag': 'O'},
 {'id': 5,
  'question': '你知道诺基亚lumia1320的主屏尺寸是多少吗？',
  'answer': '诺基亚lumia1320 ||| 主屏尺寸 |